In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score, make_scorer, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from IPython.display import clear_output
import time

# Funciones

In [2]:
def readFile(path):
    """
    Leer el archivo CSV en un DataFrame de pandas.
    """
    data = pd.read_csv(path)
    return data

def normalize(data):
    """
    Normalizar las características del DataFrame. Se asume que la columna 'category' es la etiqueta.
    """
    # Separar características y etiqueta
    X = data.drop(columns=['category'])
    y = data['category']

    # Normalizar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Crear un nuevo DataFrame con características normalizadas
    data_normalized = pd.DataFrame(X_scaled, columns=X.columns)
    data_normalized['category'] = y.values  # Añadir la columna de etiquetas

    return data_normalized


In [3]:
def train(data_normalized, alpha):
    """
    Entrenar el modelo L2 Ridge con el parámetro de regularización especificado.
    """
    # Separar características y etiqueta
    X = data_normalized.drop(columns=['category'])
    y = data_normalized['category']

    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Inicializar el modelo RidgeClassifier
    model = RidgeClassifier(alpha=alpha, solver='auto')  # Puedes ajustar el solver si es necesario

    # Entrenar el modelo
    model.fit(X_train, y_train)

    return model, X_test, y_test

In [4]:
def accuracy(X_test, y_test, model):
    """
    Calcular la precisión del modelo en el conjunto de prueba.
    """
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Ejecucion lectura datos


In [5]:
# Medir el tiempo de inicio
start_time = time.time()

In [6]:
col_names = [
    'pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport',
    'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev',
    'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco',
    'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
    'attack', 'category', 'subcategory'
]

# Definir los tipos de datos correspondientes a cada columna
col_types = {
    'pkSeqID': int, 'stime': float, 'flgs': str, 'proto': str,
    'saddr': str, 'sport': float, 'daddr': str, 'dport': float, 'pkts': int, 'bytes': int,
    'state': str,'ltime': float, 'seq': int, 'dur': float, 'mean': float, 'stddev': float, 'smac': str,
    'dmac': str, 'sum': float, 'min': float, 'max': float, 'soui': float, 'doui': float,
    'sco': float, 'dco': str, 'spkts': str, 'dpkts': str, 'sbytes': str, 'dbytes': str,
    'rate': str, 'srate': str, 'drate': str, 'attack': str, 'category': str, 'subcategory': str
}

# Definir las URLs de los archivos CSV
url_base = 'https://raw.githubusercontent.com/Meusz/FinOps/main/data/data_'
urls = [url_base + str(i) + '.csv' for i in range(1, 19)]

# Inicializar un DataFrame vacío
df_combinado = pd.DataFrame(columns=col_names)
# Convertir tipos de columnas según el diccionario col_types
df_combinado = df_combinado.astype(col_types)


# Descargar y combinar los archivos CSV en un DataFrame

for url in urls:
    clear_output()
    print(f"Ultimo URL leido:{url}")

    df = pd.read_csv(url,names=col_names,header=0)
    # Convertir 'sport' y 'dport' a tipo numérico, ignorando los errores
    df['sport'] = pd.to_numeric(df['sport'], errors='coerce')
    df['dport'] = pd.to_numeric(df['dport'], errors='coerce')

    # Llenar NaN en las columnas con un valor predeterminado, por ejemplo 0
    df['pkts'].fillna(0, inplace=True)
    df['bytes'].fillna(0, inplace=True)
    df['seq'].fillna(0, inplace=True)

    # Convertir las columnas a tipo int después de manejar NaN
    df['pkts'] = df['pkts'].astype(int)
    df['bytes'] = df['bytes'].astype(int)
    df['seq'] = df['seq'].astype(int)

    df=df.astype(col_types)
    # Combinar los DataFrames
    df_combinado = pd.concat([df_combinado, df])
    del df


# Mostrar el DataFrame combinado
clear_output()

df_combinado.drop(df_combinado[df_combinado['category'] == 'nan'].index, inplace=True)

#["flgs", "proto", "pkts", "bytes", "dur", "mean", "stddev", "sum", "min", "max", "rate", "category"]

df_combinado.loc[df_combinado["proto"] == "tcp", "proto"] = 0
df_combinado.loc[df_combinado["proto"] == "udp", "proto"] = 1
df_combinado.loc[df_combinado["proto"] == "icmp", "proto"] = 2
df_combinado.loc[df_combinado["proto"] == "arp", "proto"] = 3
df_combinado.loc[df_combinado["proto"] == "ipv6-icmp", "proto"] = 4
df_combinado.loc[df_combinado["proto"] == "igmp", "proto"] = 4
df_combinado.loc[df_combinado["proto"] == "rarp", "proto"] = 4



df_combinado.loc[df_combinado["category"] == "Reconnaissance", "category"] = 0
df_combinado.loc[df_combinado["category"] == "DoS", "category"] = 1
df_combinado.loc[df_combinado["category"] == "Normal", "category"] = 2
df_combinado.loc[df_combinado["category"] == "Theft", "category"] = 3
df_combinado.loc[df_combinado["category"] == "Reconnai", "category"] = 4

df_combinado['category'] = df_combinado['category'].astype(int)
df_combinado['proto'] = df_combinado['proto'].astype(int)




df_combinado = df_combinado.dropna(subset=["flgs", "proto", "pkts", "bytes", "dur", "mean", "stddev", "sum", "min", "max", "rate", "category"])
#df_combinado.drop(df_combinado[df_combinado['daddr'] == 'nan'].index, inplace=True)

df_combinado

,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,bytes,...,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category,subcategory
0,1,1.526344e+09,e,3,192.168.100.1,NaN,192.168.100.3,NaN,4,240,...,2,2,120,120,0.002508,0.000836,0.000836,0,2,Normal
1,2,1.526344e+09,e,0,192.168.100.7,139.0,192.168.100.4,36390.0,10,680,...,5,5,350,330,0.00619,0.002751,0.002751,0,2,Normal
2,3,1.526344e+09,e,1,192.168.100.149,51838.0,27.124.125.250,123.0,2,180,...,1,1,90,90,20.59096,0.0,0.0,0,2,Normal
3,4,1.526344e+09,e,3,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,2,Normal
4,5,1.526344e+09,e,1,192.168.100.27,58999.0,192.168.100.1,53.0,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,2,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110023,1653909,1.526963e+09,e,0,192.168.100.149,41228.0,192.168.100.5,5060.0,2,120,...,1.0,1.0,60.0,60.0,300.300293,0.0,0.0,1.0,0,OS_Fingerprint
110024,1653910,1.526963e+09,e,0,192.168.100.149,41228.0,192.168.100.5,32769.0,2,120,...,1.0,1.0,60.0,60.0,295.246521,0.0,0.0,1.0,0,OS_Fingerprint
110025,1653911,1.526963e+09,e,0,192.168.100.149,41228.0,192.168.100.5,1233.0,2,120,...,1.0,1.0,60.0,60.0,292.312195,0.0,0.0,1.0,0,OS_Fingerprint
110026,1653912,1.526963e+09,e,0,192.168.100.149,41228.0,192.168.100.5,16992.0,2,120,...,1.0,1.0,60.0,60.0,283.848999,0.0,0.0,1.0,0,OS_Fingerprint


In [7]:
df_combinado['proto'] = df_combinado['proto'].astype(int)

In [8]:
# Se eliminan las columnas innecesarias del DataFrame
df_combinado=df_combinado.drop(columns = ['pkSeqID', 'stime', 'flgs', 'ltime', 'seq', 'smac',  'dmac', 'soui', 'doui', 'sco', 'dco', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'srate', 'drate', 'attack', 'subcategory'])

# Selecciona las columnas de tipo 'object' en el DataFrame  y devuelve sus nombres
print(df_combinado.select_dtypes(include=['object']).columns)

# Calcula la cantidad de valores NaN por columna en el DataFrame
print(df_combinado.isna().sum())


# Elimina las filas donde la columna 'sport' tiene valores NaN en el DataFrame

df_combinado = df_combinado.dropna(subset=['sport','proto'])

# Elimina las filas duplicadas
df_combinado.drop_duplicates(inplace = True)

# Elimina las columnas especificadas del DataFrame
df_combinado = df_combinado.drop(columns = ['saddr', 'daddr',  'state', 'sport', 'dport'])


# Guardar el DataFrame df_combinado en un archivo CSV
df_combinado.to_csv('botnet.csv', index=False)
print(df_combinado.head())
del df_combinado

Index(['saddr', 'daddr', 'state', 'rate'], dtype='object')
proto           0
saddr           0
sport       32601
daddr           0
dport       32602
pkts            0
bytes           0
state           0
dur             0
mean            0
stddev          0
sum             0
min             0
max             0
rate            0
category        0
dtype: int64
   proto  pkts  bytes          dur      mean    stddev       sum       min  \
1      0    10    680  1453.945923  0.000028  0.000008  0.000138  0.000022   
2      1     2    180     0.048565  0.048565  0.000000  0.048565  0.048565   
4      1     4    630   569.933960  0.098505  0.080150  0.197011  0.018356   
7      1     2    172     2.500101  0.000000  0.000000  0.000000  0.000000   
8      1     2    172     2.501101  0.000000  0.000000  0.000000  0.000000   

        max      rate  category  
1  0.000042   0.00619         2  
2  0.048565  20.59096         2  
4  0.178655  0.005264         2  
7  0.000000  0.399984         2  
8

In [9]:
# Extraer el archivo CSV del ZIP y cargarlo en un DataFrame
path = 'botnet.csv'
nIter = 5
learningRate = 0.1
lambda_reg = 0.1

In [10]:
# Medir el tiempo de finalización
end_time = time.time()
# Calcular y mostrar el tiempo de ejecución
execution_time = end_time - start_time
print(f'Tiempo de ejecución: {execution_time:.2f} segundos, {execution_time/60:.2f}  minutos')

Tiempo de ejecución: 96.54 segundos, 1.61  minutos


# Ejecucion entrenamiento

In [11]:
# Medir el tiempo de inicio
start_time = time.time()

In [12]:
# Leer datos del archivo CSV
data = readFile('botnet.csv')  # Cambia 'path_to_your_file.csv' a la ruta real de tu archivo

# Normalizar los datos
data_normalized = normalize(data)

In [13]:
# Entrenar el modelo RidgeClassifier
alpha = 1.0  # Puedes ajustar el valor de alpha
model, X_test, y_test = train(data_normalized, alpha)

In [14]:
# Predecir las etiquetas en el conjunto de prueba
y_pred = model.predict(X_test)
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 91.17%


In [18]:
# Calcular el F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')  # Cambia average a 'macro', 'micro' o 'weighted' para otros casos
print(f'F1 Score: {f1:.2f}')

# Calcular el MSE (Mean Squared Error) - Aunque MSE no es común para clasificación, se puede calcular para ver el error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.2f}')

# Calcular R²
r2 = r2_score(y_test, y_pred)
print(f'R² (Coeficiente de Determinación): {r2:.2f}')

F1 Score: 0.88
Mean Squared Error (MSE): 0.10
R² (Coeficiente de Determinación): -0.06


In [ ]:
# Medir el tiempo de finalización
end_time = time.time()
# Calcular y mostrar el tiempo de ejecución
execution_time = end_time - start_time
print(f'Tiempo de ejecución: {execution_time:.2f} segundos, {execution_time/60:.2f}  minutos')

# Cross-Validation

## Ejecucion

In [ ]:
# Medir el tiempo de inicio
start_time = time.time()

In [ ]:
#definir cuantas bloques quiere dividir para cross-validation
iterations = 5
avg_acc = 0

In [ ]:
for i in range(iterations):
    model, X_test, y_test = train(data_normalized, alpha)
    # Predecir las etiquetas en el conjunto de prueba
    y_pred = model.predict(X_test)
    # Calcular la precisión
    acc = accuracy_score(y_test, y_pred)
    avg_acc += acc
    print(f"Cross-Validation con clave:{i}, the acc:{acc}")
avg_acc = avg_acc/iterations
print("averge acc:" , avg_acc)

In [ ]:
# Medir el tiempo de finalización
end_time = time.time()
# Calcular y mostrar el tiempo de ejecución
execution_time = end_time - start_time
print(f'Tiempo de ejecución: {execution_time:.2f} segundos, {execution_time/60:.2f}  minutos')

# Graficos para el informe

## funcion

In [ ]:
import matplotlib.pyplot as plt

def plot_feature_importances(model, feature_names):
    """
    Graficar las importancias de las características del modelo RidgeClassifier.
    """
    # Las importancias de las características se encuentran en coef_ del modelo Ridge
    importances = model.coef_[0]

    # Crear un DataFrame para las importancias
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # Graficar las importancias de las características
    plt.figure(figsize=(12, 8))
    plt.barh(importance_df['Feature'], importance_df['Importance'])
    plt.xlabel('Importancia')
    plt.ylabel('Características')
    plt.title('Importancia de las Características en el Modelo RidgeClassifier')
    plt.gca().invert_yaxis()
    plt.show()

def plot_learning_curve(X_train, y_train, model, alpha):
    """
    Graficar la curva de aprendizaje para el modelo RidgeClassifier.
    """
    from sklearn.model_selection import learning_curve

    train_sizes, train_scores, test_scores = learning_curve(
        model, X_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5
    )

    plt.figure(figsize=(12, 8))
    plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Puntuación de Entrenamiento', marker='o')
    plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Puntuación de Validación', marker='o')
    plt.xlabel('Tamaño del Conjunto de Entrenamiento')
    plt.ylabel('Precisión')
    plt.title(f'Curva de Aprendizaje para RidgeClassifier con alpha={alpha}')
    plt.legend()
    plt.grid()
    plt.show()


## Ejecucion

In [ ]:
# Medir el tiempo de inicio
start_time = time.time()

In [ ]:
# Graficar la importancia de las características
feature_names = X_test.columns
plot_feature_importances(model, feature_names)

In [ ]:
# Graficar la curva de aprendizaje
plot_learning_curve(X_test, y_test, model, alpha)

In [ ]:
# Medir el tiempo de finalización
end_time = time.time()
# Calcular y mostrar el tiempo de ejecución
execution_time = end_time - start_time
print(f'Tiempo de ejecución: {execution_time:.2f} segundos, {execution_time/60:.2f}  minutos')

# Analizar componentes

In [ ]:
import psutil
import subprocess
import os

# Obtener información del procesador
cpu_info = os.popen("cat /proc/cpuinfo | grep 'model name' | uniq").read().strip()
print(f'Modelo de procesador: {cpu_info}')

# Número de procesadores físicos
num_processors = psutil.cpu_count(logical=False)
print(f'Número de procesadores físicos: {num_processors}')

# Número de vCores
num_vcores = psutil.cpu_count(logical=True)
print(f'Número de vCores (procesadores lógicos): {num_vcores}')

# Capacidad de memoria
mem = psutil.virtual_memory()
total_memory_gb = mem.total / (1024 ** 3)  # Convertir bytes a GB
available_memory_gb = mem.available / (1024 ** 3)  # Convertir bytes a GB
print(f'Capacidad total de memoria RAM: {total_memory_gb:.2f} GB')
print(f'Memoria RAM disponible: {available_memory_gb:.2f} GB')

# Información del disco duro
disk_usage = psutil.disk_usage('/')
total_disk_gb = disk_usage.total / (1024 ** 3)  # Convertir bytes a GB
used_disk_gb = disk_usage.used / (1024 ** 3)    # Convertir bytes a GB
free_disk_gb = disk_usage.free / (1024 ** 3)    # Convertir bytes a GB
print(f'Capacidad total del disco duro: {total_disk_gb:.2f} GB')
print(f'Espacio utilizado del disco duro: {used_disk_gb:.2f} GB')
print(f'Espacio libre del disco duro: {free_disk_gb:.2f} GB')

# Tipo de disco duro
disk_info = os.popen("lsblk -o NAME,ROTA,TYPE,SIZE | grep '^sda'").read().strip()
print(f'Tipo de disco duro: {disk_info}')

# Información del nodo
node_info = os.uname()
print(f'Información del nodo: {node_info}')

# Información detallada del sistema
print(f'Información detallada del sistema:')
print(f'Sistema: {node_info.sysname}')
print(f'Nombre del nodo: {node_info.nodename}')
print(f'Release: {node_info.release}')
print(f'Versión: {node_info.version}')
print(f'Máquina: {node_info.machine}')

# Obtener información de la GPU
try:
    gpu_info = subprocess.check_output("nvidia-smi --query-gpu=name --format=csv,noheader", shell=True).decode('utf-8').strip()
    print(f'Modelo de GPU: {gpu_info}')
except subprocess.CalledProcessError:
    print('No se detectó GPU o NVIDIA-SMI no está instalado.')
